In [1]:
import sys
import os
from os.path import join as pj
import deepcopy
from astropy.io import fits
# from os.path import exists

_HOME_DIR = os.path.expanduser("~")    

try:
    _SPARCFIRE_DIR = os.environ["SPARCFIRE_HOME"]
    _MODULE_DIR = pj(_SPARCFIRE_DIR, "GalaxyMusic", "GalfitModule")
except KeyError:
    # print("SPARCFIRE_HOME is not set. Please run 'setup.bash' inside SpArcFiRe directory if not done so already.")
    # print("Running on the assumption that GalfitModule is in your home directory... (if not this will fail and quit!)") 
    _MODULE_DIR = pj(_HOME_DIR, "GalaxyMusic", "GalfitModule")
    
sys.path.append(_MODULE_DIR)
    
from Classes.Components import *
from Classes.Containers import *
from Functions.helper_functions import *
from sparc_to_galfit_feedme_gen import *

In [ ]:
g_prefix = "galaxy"
g_number = 0

IN_DIR  = pj(_HOME_DIR, "GalaxyMusic", "music-in")
TMP_DIR = pj(_HOME_DIR, "GalaxyMusic", "music-tmp")
OUT_DIR = pj(_HOME_DIR, "GalaxyMusic", "music-out")

run_galfit = pj(_MODULE_DIR, "galfit")

previous = False

In [ ]:
# in loop
gname = f"{g_prefix}_{g_number}"
filename = f"{pj(IN_DIR, gname)}.fits"
template = pj(_MODULE_DIR, "template.fits")
sp(f"cp {template} {filename}")
#gen_template(filename)

if not exists(pj(OUT_DIR, gname)):
    os.mkdir(pj(OUT_DIR, gname))
    
feedme = pj(OUT_DIR, gname, f"{gname}.in")

if previous:
    galaxy_feedme = previous
else:
    galaxy_feedme = FeedmeContainer(path_to_feedme = feedme)

    galaxy_feedme.header.param_values.update({
        "input_image"  : filename,
        "output_image" : pj(OUT_DIR, gname, f"{gname}.fits"),
        "optimize"     : 1,
        "region_to_fit" : (0,148,0,148)
        }
    )

disk, bulge, dict_galfit = music_to_galaxy_properties(dict_music)
dict_galfit.update({"position" : (74, 74)})

if bulge == 1.0:
    galaxy_feedme.bulge.param_values.update(dict_galfit)
elif disk == 1.0:
    galaxy_feedme.disk.param_values.update(dict_galfit)
else:
    print("something went wrong")

galaxy_feedme.update_components(bulge = bulge, disk = disk)

#print(galaxy_feedme)
process = sp(f"{run_galfit} {feedme}")

previous = deepcopy(galaxy_feedme)
#print(process.stdout)